In [19]:

import matplotlib
import pandas as pd
import numpy as np

import pickle 
import os
import re
import inspect

from collections import defaultdict
from functools import  partial
from itertools import  chain
from dataclasses import dataclass

print(pd.__version__)

VERBOSE = False

0.24.0


##### Load Nominations

ToDo
1. more than 1000 articles in duplicates?
2. Check warum die alten Archive auch geparst werden

Visualisierungen
1. Multinominierung


##### Load ends, transform it to a DataFrame and keep a set of all article names

In [30]:
df_ends.columns

Index(['idx', 'title', 'dates'], dtype='object')

# I. Summarize clean

In [11]:
# Load results of parsing monthly archives
df_FAC = pd.read_csv('./data/FAC_nomination.csv', sep=';', index_col=0, parse_dates=['nomination', 'last_comment'])  
df_FAC = df_FAC[(np.datetime64('2005')<=df_FAC.nomination) & (df_FAC.nomination<=np.datetime64('2016'))]


# Load results of parsing revision history 
with open('./res/article_dict.pkl', 'rb') as file:
    ends_dict = pickle.load(file)
    
# convert revision histotory from dictionary to pd.DataFrame
df_ends = [[i, article, dates]for i, (article, dates) in enumerate(ends_dict.items())]
df_ends = pd.DataFrame(df_ends, columns=['idx', 'title', 'dates'])

# pd.DataFrame with min/max date
df_ends_min = [[i, article, max(dates), min(dates)] for i, (article, dates) in enumerate(ends_dict.items())]
df_ends_min = pd.DataFrame(df_ends_min, columns=['idx', 'title', 'first', 'last'])

# clean article names (min-version)
# it's possible to increase  number of matching articles, by preprocessing
df_FAC['title'] = df_FAC.title.str.replace('/archive\d', '') # These are actually unique nominations
df_ends['title'] = df_ends.title.str.replace('/archive\d', '') # These likely not
df_ends_min['title'] = df_ends_min.title.str.replace('/archive\d', '')

# This will produce duplicates,
#   1) append all dates
df_ends['dates'] = df_ends.groupby('title').agg({'dates' :'sum'})
#df_ends.reset_index(inplace=True)

##  2) chose min, max
agg_funcs = {'first': 'min', 
             'last' : 'max'}
df_ends_min = df_ends_min.groupby('title').agg(agg_funcs)


n_FAC = len(df_FAC)
n_ends = len(df_ends)

# We only keep the first nomination (simplification)
df_FAC['has_duplicate'] = df_FAC.duplicated(subset='title', keep=False)
print('duplicates:')
print(f'multiple nominations: {sum(df_FAC.has_duplicate)}')
df_FAC = df_FAC.loc[~df_FAC.sort_values('nomination').duplicated(subset='title', keep='first'),]

# merge
df_merge = pd.merge(df_FAC, df_ends, on='title')
n_FAC_nd = len(df_FAC)
print(f'number of nominations: {n_FAC:>20} \nartilcles in ends: {n_ends:>20}'
      f'\nunique articles in nominations: {n_FAC_nd:10} '
      f'\narticles in merged data Frame: {len(df_merge):15}')

df_FAC.to_csv('./tmp/FAC_merge.csv', sep=';')

duplicates:
multiple nominations: 1845
number of nominations:                 4966 
artilcles in ends:                10850
unique articles in nominations:       3885 
articles in merged data Frame:            4886


# II. Decide End

In [33]:


with open('./res/article_dict.pkl', 'rb') as file:
    ends_dict = pickle.load(file)

# Set up dict {article : np.array[1, len(dates)]}df_FAC = pd.read_csv('./tmp/FAC_merge.csv', sep=';', parse_dates=[3,4], index_col=0)
res = defaultdict(list)
for article, dates in ends_dict.items():
    article = re.sub('/archive\d', '', article)
    dates = np.array([np.datetime64(date) for date in dates])
    res[article].append(dates)
ends_dict = res

# get set of titles
art_set = set(ends_dict.keys())

# read df_FAC (post I.)

df_FAC = df_FAC.loc[df_FAC.title.isin(art_set),]
print(df_FAC.dtypes)

# convert pd.DataFrame to 
FAC_dict = {k: np.datetime64(v) for k,v in df_FAC[['title', 'last_comment']].values}
print(len(FAC_dict), len(ends_dict))

new_dict = {}
count =0
for article, last_date in FAC_dict.items():
    two_weeks_later = last_date + np.timedelta64(2, 'W')
    dates = list(chain.from_iterable(ends_dict[article]))   # just to be sure list is falttened
    dates = [date for date in dates if date < two_weeks_later]
    # print(two_weeks_later, type(dates), type(dates[0]), type(two_weeks_later))
    #new_date = np.max(dates) if dates else last_date        
    if dates:
        new_date  = max(dates)
    else:
        new_date = last_date
        print(f'No date (< 2W) found for article: {article}')
    new_dict[article] = new_date

idx                       int64
title                    object
nomination       datetime64[ns]
last_comment     datetime64[ns]
has_duplicate              bool
dtype: object
3773 9645
No date (< 2W) found for article: Taylor Swift
No date (< 2W) found for article: Jack Parsons (rocket engineer)
No date (< 2W) found for article: Potential cultural impact of extraterrestrial contact
No date (< 2W) found for article: Fluorine
No date (< 2W) found for article: Ashley Tisdale
No date (< 2W) found for article: Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan
No date (< 2W) found for article: iPod
No date (< 2W) found for article: Pixies (band)
No date (< 2W) found for article: West Bengal
No date (< 2W) found for article: Paul Martin
No date (< 2W) found for article: Stanley Kubrick
No date (< 2W) found for article: Moncton
No date (< 2W) found for article: Freemasonry
No date (< 2W) found for article: Stonewall riots
No date (< 2W) found for article: Earth

numpy.datetime64('2005-01-14T15:26:07.000000')

In [15]:
FAC_dict


{}

In [ ]:
i = 0
for article, dates in art_dict.items():
    dates = np.array([np.datetime64(date) for date in dates])
    print(f'min:{min(dates)} max:{max(dates)}')
    p1 = dates[(dates- dates[0]) > np.timedelta64(30, 'D')]
    p2 = dates[~((dates- dates[0]) > np.timedelta64(30, 'D'))]
    print(article)
    print(len(p1), len(p2))
    if i >= 1:
        break
    i+=1

from pprint import pprint
pprint(res['Belgium national football team'])

In [4]:
df_FAC = pd.read_csv('./data/FAC_nomination.csv', sep=';', index_col=0, parse_dates=['nomination', 'last_comment'])  
d = df_FAC.groupby('title').count().sort_values('idx')
len(d.loc[d.idx >= 4])

69

In [18]:
with open('./res/article_dict.pkl', 'rb') as file:
    ends_dict = pickle.load(file)
    
df_ends = [[i, article, set(dates)]for i, (article, dates) in enumerate(ends_dict.items())]
df_ends = pd.DataFrame(df_ends)

df_ends



,0,1,2
0,0,Same-sex marriage,"{2003-09-01 00:17:27, 2003-10-02 06:17:00, 200..."
1,1,MKULTRA,"{2003-09-01 00:17:27, 2003-10-02 06:17:00, 200..."
2,2,Aztalan State Park,"{2003-09-01 00:17:27, 2003-10-02 06:17:00, 200..."
3,3,Provinces of Thailand,"{2003-11-30 10:39:40, 2003-10-15 01:15:15, 200..."
4,4,Lawrence v. Texas,"{2003-10-15 01:15:15, 2003-09-01 00:17:27, 200..."
5,5,Damascus steel,"{2003-09-28 06:00:19, 2003-08-28 23:53:05, 200..."
6,6,White Rose,"{2003-09-28 06:00:19, 2003-08-28 23:53:05, 200..."
7,7,Chariot racing,"{2003-09-28 06:00:19, 2003-08-28 23:53:05, 200..."
8,8,Schizophrenia,"{2005-02-12 17:40:04, 2005-02-24 18:11:06, 200..."
9,9,Beach Cricket,{2003-08-04 06:59:33}
